In [1]:
conda install tensorflow

Solving environment: ...working... done


Note: you may need to restart the kernel to use updated packages.# All requested packages already installed.






==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda




In [2]:
import tensorflow as tf
from keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [3]:
def transformer_block(embed_dim, num_heads, ff_dim, dropout_rate):
    att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    ffn = Sequential(
        [Dense(ff_dim, activation="relu"),
         Dense(embed_dim), ]
    )
    layernorm1 = LayerNormalization(epsilon=1e-6)
    layernorm2 = LayerNormalization(epsilon=1e-6)
    dropout1 = Dropout(dropout_rate)
    dropout2 = Dropout(dropout_rate)
    
    def call(inputs, training):
        attn_output = att(inputs, inputs)
        attn_output = dropout1(attn_output, training=training)
        out1 = layernorm1(inputs + attn_output)
        ffn_output = ffn(out1)
        ffn_output = dropout2(ffn_output, training=training)
        return layernorm2(out1 + ffn_output)
    
    return call

In [21]:
# Load training data
train_df = pd.read_csv('../data/tone_train.csv')
x_train = np.array(train_df['numerical_sequence'].dropna().apply(lambda x: [int(i) for i in x.split(',')]).tolist())
y_train = np.array(train_df['revised_BERT_sentiment_score'].dropna())

In [22]:
# Load test data
test_df = pd.read_csv('../data/tone_test.csv')
x_test = np.array(test_df['numerical_sequence'].apply(lambda x: [int(i) for i in x.split(',')]).tolist())
y_test = np.array(test_df['revised_BERT_sentiment_score'])

In [23]:
# Pad sequences to a fixed length
maxlen = 227 # assuming the maximum sequence length is 200
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen, truncating='post')
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen, truncating='post')

In [24]:
# Training and validation split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [26]:
# Concatenate x_train and x_test along the appropriate axis (assuming axis=0)
x_combined = np.concatenate((x_train, x_test), axis=0)
# Get the maximum value across all elements in the combined array
vocab_size = np.max(x_combined) + 1
vocab_size

38217

In [27]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', 
                               # Number of epochs to wait for improvement
                               patience=3,  
                               verbose=1, 
                               # Restore the weights of the best epoch
                               restore_best_weights=True)  

In [29]:
# define model architecture
max_seq_length = x_train.shape[1]
vocab_size = vocab_size
embedding_dim = 16
num_heads = 2
ff_dim = 32
dropout_rate = 0.3

inputs = Input(shape=(max_seq_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)
transformer_block_fn = transformer_block(embed_dim=embedding_dim, 
                                         num_heads=num_heads, 
                                         ff_dim=ff_dim,
                                        dropout_rate=dropout_rate)
transformer_block = transformer_block_fn(embedding_layer, training=True)
pooling_layer = GlobalAveragePooling1D()(transformer_block)
dropout_layer = Dropout(dropout_rate)(pooling_layer)
outputs = Dense(units=5, activation='softmax')(dropout_layer)

model = Model(inputs=inputs, outputs=outputs)

# compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
model.fit(x_train, y_train, batch_size=16, epochs=10, validation_data=(x_val, y_val),
          callbacks=[early_stopping])

# save the trained model
model.save('../models/transformer_classification_model.h5')

TypeError: 'KerasTensor' object is not callable

In [30]:
model = load_model('../models/transformer_classification_model.h5')

Do more epochs and get better accuracy
Evaluate model

In [31]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(x_val, y_val)
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

78/78 [==============================] - 2s 18ms/step - loss: 0.7625 - accuracy: 0.6787
Validation loss: 0.7625349760055542
Validation accuracy: 0.6787003874778748
